In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import permutation as perm
%matplotlib inline

import keras.backend as K
from keras.datasets import cifar100 as c100
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import yolo.config as cfg
from yolo.process import preprocess

Using TensorFlow backend.


In [2]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [4]:
print(K.image_data_format())
print(K.image_dim_ordering())
inp_w, inp_h, inp_c = cfg.inp_size
batch_size = 32

channels_last
tf


In [5]:
def prepare_output_data(y_train, y_test):
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    return y_train, y_test

In [6]:
def cifar_generator():
    """
    X_data : (data_size, 32, 32, 3)
    y_label : (data_size, 100)
    batch_size : batch size
    """
    (X_train, y_train), (X_test, y_test) = c100.load_data() # Load Data
    y_train, y_test = prepare_output_data(y_train, y_test) # prepare y_label 
    data_size = len(X_train)
    batch_iter_per_epoch = int(data_size/batch_size)
    shuffle_idx = perm(np.arange(len(X_train)))
    print("*"*30)
    print("Data Size : {}".format(data_size))
    print("Batch Size : {}".format(batch_size))
    print("Batch iterations per Epoch : {}".format(batch_iter_per_epoch))
    print("*"*30)
    print(shuffle_idx[0:10])
    for b in range(batch_iter_per_epoch):
        batch_features = np.zeros((batch_size, inp_w, inp_h, inp_c))
        batch_labels = np.zeros((batch_size, 100))
        
        for b_i, i in enumerate(range(b*batch_size, b*batch_size + batch_size)):
            # choose random index in features
            batch_features[b_i] = preprocess(X_train[shuffle_idx[i]], color_type='RGB')
            batch_labels[b_i] = y_train[shuffle_idx[i]]
        yield batch_features, batch_labels
    
    print("Done Generator")

In [7]:
def make_model(is_freeze=False, activation_type='relu'):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(inp_w, inp_h, inp_c),padding='same', 
                            activation=activation_type, trainable=not is_freeze, name='conv1'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32,(3,3), padding='same', 
                            activation=activation_type, trainable=not is_freeze, name='conv2'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(64,(3,3), padding='same', 
                            activation=activation_type, trainable=not is_freeze, name='conv3'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(128,(3,3), padding='same', 
                            activation=activation_type, trainable=not is_freeze, name='conv4'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(256,(3,3), padding='same', 
                            activation=activation_type, trainable=not is_freeze, name='conv5'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(512,(3,3), padding='same', 
                            activation=activation_type, trainable=not is_freeze, name='conv6'))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Conv2D(1024,(3,3), padding='same', activation=activation_type, trainable=not is_freeze, name='conv7'))
    model.add(Conv2D(1024,(3,3), padding='same', activation=activation_type, trainable=not is_freeze, name='conv8'))
    model.add(Conv2D(1024,(3,3), padding='same', activation=activation_type, trainable=not is_freeze, name='conv9'))
    model.add(Flatten())
    model.add(Dense(256, name='dense1', activation=activation_type))
    model.add(Dense(4096, name='dense2', activation=activation_type))
    model.add(Dense(100, name='softmax', activation='softmax'))
    
    return model

In [10]:
model = make_model(is_freeze=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 448, 448, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 224, 224, 16)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 224, 224, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv3 (Conv2D)               (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv4 (Conv2D)               (None, 56, 56, 128)       73856     
__________

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
history = model.fit_generator(cifar_generator(), 
                    steps_per_epoch=1562, 
                    nb_epoch=2, 
                    verbose=1,
                    validation_data=None, 
                    class_weight=None,
                    workers=1)

/Users/myZZUNG/venvOpenCV3/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1562, validation_data=None, verbose=1, class_weight=None, workers=1, epochs=2)`


Epoch 1/2
******************************
Data Size : 50000
Batch Size : 32
Batch iterations per Epoch : 1562
******************************
[35434 32406 36814 21697 44165 17445 16948 35995  2143 49534]
   3/1562 [..............................] - ETA: 32685s - loss: 4.9425 - acc: 0.0104    

KeyboardInterrupt: 

In [9]:
for i, (x_batch, y_batch) in enumerate(cifar_generator()):
    if i % 100 == 0:
        print("{} done..".format(i))

******************************
Data Size : 50000
Batch Size : 32
Batch iterations per Epoch : 1562
******************************
[47275 39241 44355 32672 18493  1810 29567 19884 14608  4119]
0 done..
100 done..


KeyboardInterrupt: 